In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget

In [ ]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter

Building a simple SEIR model (based on https://www.frontiersin.org/articles/10.3389/fams.2023.1124897/full#T1)

In [ ]:
def build_model():
    m = CompartmentalModel([0,100], ["S","E","I","R"],"I",ref_date=datetime(2023,1,1))
    m.set_initial_population({"S": 166198.0, "E": 16797.0, "I": 481.0, "R": 487.0})
    m.add_importation_flow("recruitment", num_imported=33595, dest="S", split_imports=True) #A
    m.add_universal_death_flows("universal_death", death_rate=0.143) #mu
    m.add_infection_density_flow("infection", Parameter("contact_rate"),"S","E") #alpha I
    m.add_transition_flow("partial_vax", 0.2522,"S","E") #V_p
    m.add_transition_flow("full_vax", 0.1517,"S","R") #V_f
    m.add_transition_flow("progression", Parameter("progression"),"E","I") #delta
    m.add_transition_flow("recovery", 0.09,"I","R") #gamma
    m.add_death_flow("disease_death", 0.1595,"I") #mu_1
    m.request_output_for_flow("infection", "infection")
    m.request_output_for_flow("recovery", "recovery")
    m.request_output_for_flow(
        "progressions",
        "progression"
    )
    return m

In [ ]:
m = build_model()

In [ ]:
m.get_input_parameters()

In [ ]:
parameters = {"contact_rate": 0.00001, "progression": 1.01}

In [ ]:
m.run(parameters)

In [ ]:
m.get_outputs_df().plot()

In [ ]:
m.get_derived_outputs_df().plot()

In [ ]:
ndata = m.get_derived_outputs_df()["progressions"]
ndata = ndata[:30]

In [ ]:
ndata.plot()

Calibration

In [ ]:
# Targets represent data we are trying to fit to
from estival import targets as est

# We specify parameters using (Bayesian) priors
from estival import priors as esp

# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

In [ ]:
targets = [
    est.NormalTarget("progressions", ndata, np.std(ndata) * 0.1)
]

In [ ]:
priors = [
    esp.UniformPrior("contact_rate", (0.0,0.5)),
    esp.UniformPrior("progression", (0.5,1.5))
]

In [ ]:
defp = {"contact_rate": 0.00002, "progression": 0.95}

In [ ]:
bcm = BayesianCompartmentalModel(m, defp, priors, targets)
from estival.wrappers import pymc as epm
import pymc as pm

In [ ]:
with pm.Model() as model:

    # This is all you need - a single call to use_model
    variables = epm.use_model(bcm)

    # The log-posterior value can also be output, but may incur additional overhead
    # Use jacobian=False to get the unwarped value (ie just the 'native' density of the priors
    # without transformation correction factors)
    # pm.Deterministic("logp", model.logp(jacobian=False))

    # Now call a sampler using the variables from use_model
    # In this case we use the Differential Evolution Metropolis sampler
    # See the PyMC docs for more details
    idata = pm.sample(step=[pm.DEMetropolis(variables)], draws=4000, tune=0,cores=4,chains=4)

Visualizing Outputs

In [ ]:
import arviz as az

In [ ]:
az.summary(idata)

In [ ]:
az.plot_trace(idata, figsize=(16,3.2*len(idata.posterior)),compact=False);#, lines=[("m", {}, mtrue), ("c", {}, ctrue)]);

In [ ]:
az.plot_posterior(idata);

In [ ]:
Obtaining Likelihood

In [ ]:
from estival.sampling.tools import likelihood_extras_for_idata

In [ ]:
likelihood_df = likelihood_extras_for_idata(idata, bcm)

In [ ]:
ldf_pivot = likelihood_df.reset_index(level="chain").pivot(columns=["chain"])

ldf_pivot["logposterior"].plot()

In [ ]:
# Sort this DataFrame by logposterior to obtain the MAP index
ldf_sorted = likelihood_df.sort_values(by="logposterior",ascending=False)

# Extract the parameters from the calibration samples
map_params = idata.posterior.to_dataframe().loc[ldf_sorted.index[0]].to_dict()

map_params

In [ ]:
bcm.loglikelihood(**map_params), ldf_sorted.iloc[0]["loglikelihood"]

In [ ]:
map_res = bcm.run(map_params)

In [ ]:
variable = "progressions"

pd.Series(map_res.derived_outputs[variable]).plot(title = f"{variable} (MLE)")
bcm.targets[variable].data.plot(style='.');

Uncertainty sampling

In [ ]:
sample_idata = az.extract(idata, num_samples = 400)
samples_df = sample_idata.to_dataframe().drop(columns=["chain","draw"])

In [ ]:
from estival.utils.parallel import map_parallel

In [ ]:
def run_sample(idx_sample):
    idx, params = idx_sample
    return idx, bcm.run(params)

In [ ]:
sample_res = map_parallel(run_sample, samples_df.iterrows(), n_workers=4)

In [ ]:
import xarray as xr

In [ ]:
xres = xr.DataArray(np.stack([r.derived_outputs for idx, r in sample_res]),
                    dims=["sample","time","variable"])
xres = xres.assign_coords(sample=sample_idata.coords["sample"],
                          time=map_res.derived_outputs.index, variable=map_res.derived_outputs.columns)

In [ ]:
# Set some quantiles to calculate
quantiles = (0.01,0.05,0.25,0.5,0.75,0.95,0.99)

# Generate a new DataArray containing the quantiles
xquantiles = xres.quantile(quantiles,dim=["sample"])

In [ ]:
# Extract these values to a pandas DataFrame for ease of plotting

uncertainty_df = xquantiles.to_dataframe(name="value").reset_index().set_index("time").pivot(columns=("variable","quantile"))["value"]

In [ ]:
variable = "progressions"

fig = uncertainty_df[variable].plot(title=variable,alpha=0.7)
pd.Series(map_res.derived_outputs[variable]).plot(style='--')
bcm.targets[variable].data.plot(style='.',color="black", ms=3, alpha=0.8);